In [11]:
        try:
            from .preprocessors import preprocessor, JoinSource
        except:
            from preprocessors import preprocessor, JoinSource
        from nbformat.v4 import new_markdown_cell, new_code_cell, new_notebook, new_output, nbformat_schema
        from nbformat import NotebookNode
        from mistune import Markdown, Renderer, preprocessing
        __all__ = []

In [12]:
        def codespan(text, code: str, lang="""""", markdown="""""", SEP='`'):
            return (*text.split(SEP, 2), SEP)

        def block_code(text, code: str, lang: str, block="""""", markdown="""""", SEP='```'):
            for line in filter(bool, code.splitlines()):
                leading, text = text.split(line, 1)
                if not block:
                    if SEP in leading:
                        markdown, leading = leading.split(SEP+lang, 1)
                    else:
                        (*markdown, leading), SEP = leading.splitlines(), ""
                block += leading + line
            return markdown, block, text, SEP
        
        blocks = {'codespan': codespan, 'block_code': block_code}

In [17]:
        class Notebook(Markdown):
            def render(self, body, metadata={}, outputs=[]):
                if isinstance(body, NotebookNode):
                    outputs, body = body.get('outputs'), body.get('source')
                body, cells, self.renderer.blocks = preprocessing(body), [], []
                
                # Split code by hand into markdown and code to maintain indents.
                for code, lang, type in super().render(body) and self.renderer.blocks:
                    markdown, block, body, SEP = blocks[type](body, code, lang)
                    '\n'.join(markdown).strip() and cells.append(new_markdown_cell(markdown, metadata=metadata))
                    if block.strip():
                        cells.append(new_code_cell(block, metadata=metadata))
                        cells[-1]['metadata'].update({'lang': lang, 'sep': SEP, 'src': type})
                    else:
                        cells.append(new_markdown_cell(block))
                nb = new_notebook(cells=body and cells.append(new_markdown_cell(body)) or cells)
                nb, resources = JoinSource().preprocess(nb, {})
                nb, resources = IndentInline().preprocess(nb, {})
                return nb

        class CodeCell(Renderer):
            def block_code(self, code: str, lang='') -> str:
                return self.blocks.append((code, lang or '', 'block_code')) or super().block_code(code, lang)

            def codespan(self, code: str, lang='') -> str:
                return self.blocks.append((code, lang, 'codespan')) or super().codespan(code)

        CodeCell.blocks = []
        
        renderer = Notebook(renderer=CodeCell(), parse_block_html=False)

In [18]:
        @preprocessor
        def Explode(self, nb, resources):
            cells = []
            for cell in nb['cells']:
                if cell['cell_type']=='code':
                    new = renderer.render(cell, cell['metadata'])
                    cells.extend(new['cells'])
                else:
                    cells.append(cell)
            nb.update(cells=cells) or nb 
            nb, resources = IndentInline().preprocess(nb, {})
            return nb, resources

In [16]:
        @preprocessor
        def IndentInline(self, nb, resources):
            codes = [_ for _ in nb['cells'] if _['cell_type']=='code']
            if len(codes) > 1:
                for i, cell in enumerate(codes):
                    if cell['cell_type']=='code' and 'src' in cell['metadata'] and cell['metadata']['src']=='codespan':
                        lines = codes[i-1 if i > 0 else i+1]['source'].splitlines()
                        del cell['metadata']['src']
                        while lines and not lines[-1].strip():
                            lines.pop()
                        if lines:
                            cell['source'] = ' '*(len(lines[-1]) - len(lines[-1].lstrip()))+cell['source']
            return nb, resources